In [66]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [67]:
X_features=pd.read_csv("x_features.csv").drop(["Unnamed: 0"],axis=1)

In [68]:
y=pd.read_csv("y_features.csv")

In [69]:
y_features=pd.DataFrame(y.iloc[0]).drop(["Unnamed: 0"],axis=0)

In [72]:
y_train=y_features[0:8000]
y_val=y_features[8000:9000]
y_test=y_features[9000:]
for i in tqdm(range(120)):
    y_val[y_val[0]==np.unique(y_train.values)[i]]=i
    y_test[y_test[0]==np.unique(y_train.values)[i]]=i
    y_train[y_train[0]==np.unique(y_train.values)[i]]=i

  0%|          | 0/120 [00:00<?, ?it/s]/home/aman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/aman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/aman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [58]:
from sklearn.ensemble import RandomForestClassifier

In [92]:
model=RandomForestClassifier()
model.fit(X_train.values,y_train.values)

/home/aman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [78]:
model.score(X_train.values,y_train.values)

0.99725

In [80]:
y_val_pred=model.predict(X_val.values)

In [81]:
model.score(X_val.values,y_val.values)

0.545

In [82]:
model.score(X_test.values,y_test.values)

0.5490998363338789

In [89]:
from sklearn.model_selection import RandomizedSearchCV
from time import time

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

param_dist = {"max_depth": range(100),
              "max_features": range(1, 100),
              "min_samples_split": range(2, 100),
              "min_samples_leaf": range(1, 100),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 652.87 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.584 (std: 0.007)
Parameters: {'bootstrap': False, 'min_samples_leaf': 9, 'max_features': 40, 'criterion': 'gini', 'min_samples_split': 37, 'max_depth': 86}

Model with rank: 2
Mean validation score: 0.562 (std: 0.004)
Parameters: {'bootstrap': False, 'min_samples_leaf': 16, 'max_features': 61, 'criterion': 'entropy', 'min_samples_split': 20, 'max_depth': 62}

Model with rank: 3
Mean validation score: 0.516 (std: 0.001)
Parameters: {'bootstrap': True, 'min_samples_leaf': 27, 'max_features': 62, 'criterion': 'gini', 'min_samples_split': 67, 'max_depth': 58}



In [91]:
model2=RandomForestClassifier(bootstrap=False, min_samples_leaf=9, max_features=40, criterion='gini',min_samples_split=37, max_depth=86)
model2.fit(X_train.values,y_train.values)

/home/aman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=86, max_features=40, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=9, min_samples_split=37,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [93]:
model2.score(X_train.values,y_train.values)

0.875125

In [95]:
model2.score(X_val.values,y_val.values)

0.621

In [96]:
model2.score(X_test.values,y_test.values)

0.6211129296235679